In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [25]:
XML_input = input() 

 E:\PMIDsMinimalExampleParsingXMLs\200.txt


In [3]:
DF_output = input()

  E:\PMIDsMinimalExampleParsingXMLs.csv


In [26]:
tree = ET.parse(XML_input)
root = tree.getroot()

In [43]:
data = []
for node in root:
    article_data = {}
    if node.tag == 'PubmedArticle':
        article_data['Type'] = 'Article'
        
        pmid_node = node.find('./MedlineCitation/PMID')
        article_data['PMID'] = pmid_node.text.strip() if pmid_node is not None else None

        journal_node = node.find('./MedlineCitation/Article/Journal/Title')
        article_data['Journal'] = journal_node.text.strip() if journal_node is not None else None

        title_node = node.find('./MedlineCitation/Article/ArticleTitle')
        article_data['Title'] = title_node.text.strip() if title_node is not None else None

        abstract_node = node.find('./MedlineCitation/Article/Abstract')
        if abstract_node is not None:
            abstract_text = ''
            for elem in abstract_node.iter():
                if elem.text:
                    abstract_text += elem.text.strip() + ' '
            article_data['Abstract'] = abstract_text.strip()
        else:
            article_data['Abstract'] = None
        
        authors = []
        for author in node.findall('./MedlineCitation/Article/AuthorList/Author'):
            author_info = {}
            forename_node = author.find('./ForeName')
            lastname_node = author.find('./LastName')
            if forename_node is not None and lastname_node is not None:
                author_info['Name'] = forename_node.text.strip() + ' ' + lastname_node.text.strip()
            else:
                continue

            affiliations = []
            for affiliation in author.findall('./AffiliationInfo/Affiliation'):
                affiliations.append(affiliation.text.strip())

            author_info['Affiliation'] = affiliations
            authors.append(author_info)

        article_data['Authors'] = authors

        mesh_headings = []
        for mesh_heading in node.findall('./MedlineCitation/MeshHeadingList/MeshHeading'):
            descriptor_name = mesh_heading.find('./DescriptorName').text.strip()
            mesh_headings.append(descriptor_name)
        article_data['MeshHeadings'] = ', '.join(mesh_headings) if mesh_headings else None

        chemicals = []
        for chemical in node.findall('./MedlineCitation/ChemicalList/Chemical'):
            chemical_name = chemical.find('./NameOfSubstance').text.strip()
            chemicals.append(chemical_name)
        article_data['Chemicals'] = ', '.join(chemicals) if chemicals else None

        publication_types = []
        for publication_type in node.findall('./MedlineCitation/Article/PublicationTypeList/PublicationType'):
            publication_types.append(publication_type.text.strip())
        article_data['PublicationTypes'] = ', '.join(publication_types) if publication_types else None

        pub_date_node = node.find('./MedlineCitation/Article/Journal/JournalIssue/PubDate/Year')
        if pub_date_node is not None:
            article_data['PublicationDate'] = pub_date_node.text.strip()
        else:
            article_data['PublicationDate'] = None

        language_node = node.find('./MedlineCitation/Article/Language')
        article_data['Language'] = language_node.text.strip() if language_node is not None else None

        keywords = []
        for keyword in node.findall('./MedlineCitation/KeywordList/Keyword'):
            keywords.append(keyword.text.strip())
        article_data['Keywords'] = ', '.join(keywords) if keywords else None
        
    
    elif node.tag == 'PubmedBookArticle':
        article_data['Type'] = 'Book Article'

        pmid_node = node.find('./BookDocument/PMID')
        article_data['PMID'] = pmid_node.text.strip() if pmid_node is not None else None

        publisher_node = node.find('./BookDocument/Book/Publisher/PublisherName')
        article_data['Publisher'] = publisher_node.text.strip() if publisher_node is not None else None
    else:
        continue

    
    data.append(article_data)

    

    

    

    

    

    

    

In [44]:
df = pd.DataFrame(data)

In [45]:
df

,Type,PMID,Publisher,Journal,ArticleTitle,Abstract,Authors,MeshHeadings,Chemicals,PublicationTypes,PublicationDate,Language,Keywords
0,Book Article,20301606,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Book Article,20301607,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Book Article,20301608,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Book Article,20301611,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Book Article,20301613,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Book Article,24624460,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,Book Article,24624461,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Article,24625734,NaN,Prehospital and disaster medicine,"Disaster Research/Evaluation Frameworks, Part ...",The goals of conducting disaster research are ...,"[{'Name': 'Marvin L Birnbaum', 'Affiliation': ...","Capacity Building, Disaster Planning, Disaster...",None,"Journal Article, Review, Retracted Publication",2022,eng,"Conceptual Framework, Disaster, Disaster Logic..."
198,Article,24626145,NaN,International journal of offender therapy and ...,Daily Automated Telephone Assessment and Inter...,This randomized trial evaluates whether automa...,"[{'Name': 'Claes Andersson', 'Affiliation': ['...","Adult, Community Integration, Criminals, Feedb...",None,"Journal Article, Randomized Controlled Trial",2020,eng,"Interactive Voice Response (IVR), brief interv..."


In [46]:
len(df)

200

In [47]:
df.loc[df["Journal"].isnull()]

,Type,PMID,Publisher,Journal,ArticleTitle,Abstract,Authors,MeshHeadings,Chemicals,PublicationTypes,PublicationDate,Language,Keywords
0,Book Article,20301606,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Book Article,20301607,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Book Article,20301608,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Book Article,20301611,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Book Article,20301613,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Book Article,24555207,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,Book Article,24575447,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Book Article,24624459,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,Book Article,24624460,"University of Washington, Seattle",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv(DF_output)